In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from requests.exceptions import ConnectionError
import time
import re

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# =========================================== 
# BANKI.RU
# =========================================== 


for i in range(36, 60):
    df0 = pd.DataFrame()
    print(i, pd.Timestamp.now())
    url = f'https://www.banki.ru/services/responses/bank/sberbank/?page=' + str(i) + '&is_countable=on'
    headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36",
                "Accept-Encoding": "*",
                "Connection": "keep-alive"
            }
    response = requests.get(url, headers=headers, verify=False)
    time.sleep(0.2)
    dom = bs(response.text, 'html.parser')
    links = dom.find_all('a', {'class': 'link-simple'})
    links = [link for link in links if 'data-gtm-click' in str(link)]
    time.sleep(0.2)
    for link in links:
        title = str(link.text).strip()
        
        res_link = link.get('href')
        time.sleep(0.1)
        p = f'https://www.banki.ru' + str(res_link)
 
        rsp = requests.get( p, headers=headers, verify=False)
        time.sleep(0.3)
        dom2 = bs(rsp.text, 'html.parser')  
        try:
            date = re.findall('agentAnswerDate[\s":]*\S*\s*.{8}', str(dom2))[0].replace('agentAnswerDate":"', '')

            txt = dom2.find('div', {'class': 'markdown-inside--list-type_circle-fill'}).getText().strip().replace('\t5', '').replace('\t', '').replace('\n', '')

            new_row = {'Сайт': 'www.banki.ru',  
                       'Дата': date, 
                       'Заголовок': title, 
                       'Основной текст жалобы': txt}

            df0 = df0.append(new_row, ignore_index=True)
        except:
            continue

    df0.to_excel(r'C:\Users\19586955\Documents\Morzavchenkova\python\parse\banki' + str(i) + '.xlsx', index=False)
        
    